In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from nba_api.stats.endpoints import leaguedashteamstats, playerawards

fig_DIR = "../figs/players/"
box_DIR = "../data/box/"
img_DIR_P = "../data/images/players/"

pID_dict = get_pID_pbp()
player_dict = get_players_pbp()
teams_dict, teams_list = get_teams()

## All-NBA: bbref

In [ ]:
url = "https://www.basketball-reference.com/awards/all_league.html"
df1 = pd.read_html(url)
df1 =df1[0]
df1.columns = ["Season","Lg","Tm","Voting",1,2,3,4,5]

In [ ]:
season_start = 2010
season_end = 2024
seasonsi = np.arange(season_start, season_end, 1)
seasons = list(map(lambda x: get_ss(x),seasonsi))
sdict = dict(zip(seasons,seasonsi))

In [ ]:
df2 = df1[df1["Season"].isin(seasons)]
df2 = df2[["Season","Tm",1,2,3,4,5]]
df2["Tm"] = "All-NBA " + df2["Tm"]
df3 = pd.melt(df2, id_vars=["Season","Tm"], value_vars=[1,2,3,4,5])
df3 = df3.drop(columns="variable")
df3 = df3.rename(columns={"value":"Player","Tm":"Award"})
df3["Player"] =df3["Player"].str[:-2]
df3["pID"] = df3["Player"].map(pID_dict)
df3.loc[df3["pID"].isna(),"pID"] = df3.loc[df3["pID"].isna(),"Player"].apply(lambda x: get_missing_pId(x, player_dict))
df3["pID"] = df3["pID"].astype(int)
pIDs = df3["pID"].unique()

In [ ]:
dfa = []
for id in tqdm(pIDs):
    stats = playerawards.PlayerAwards(player_id=id)
    df4 = stats.get_data_frames()[0]
    dfa.append(df4)
    time.sleep(1)
df5 = pd.concat(dfa)

In [ ]:
df6 = df5.query("DESCRIPTION == 'All-NBA'")
cols = ['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'SEASON', 'TEAM']
df6 = df6[cols]
df6["FIRST_NAME"] = df6["FIRST_NAME"] + " " + df6["LAST_NAME"]
df6 = df6.rename(columns={"FIRST_NAME":"NAME"})
df6 = df6.drop(columns={"LAST_NAME"})
df6 = df6.sort_values(["PERSON_ID","SEASON"])
df6g = df6.groupby(["PERSON_ID","NAME"])
keys = list(df6g.groups)


In [ ]:
dfb = []
for key in keys:
    df7 = df6g.get_group(key)
    df7["TEAM2"] = df7["TEAM"].shift(1)
    df7 = df7.dropna()
    dfb.append(df7)
df7 = pd.concat(dfb)
df7 = df7.reset_index(drop=True) 
df7["Change"] = df7["TEAM"] != df7["TEAM2"]

In [ ]:
df8 = df7[df7["Change"]]

In [ ]:
df9 = df8.groupby(["SEASON"])["NAME"].agg("count")
df9 = df9.reset_index()
df9.columns = ["Season","Players"]
df9["Season"] = df9["Season"].map(sdict)
# df9["Season"] = df9["Season"].str[] 

In [ ]:
from plotnine import geom_col, position_stack

In [ ]:
data = df9
p = (
    ggplot(data)
    + aes(x="Season",y="Players",group=0)
    + geom_col(show_legend=False, position=position_stack(reverse=True),color="white")
    + theme_idv
    + labs(
        title = "Number of Star Players Changing Teams",
        subtitle = "All-NBA Players Changing Teams after Making All-NBA the previous season",
        caption="bsky:@sradjoker.cc | x:@sradjoker | source: nba.com/stats",
    )
    # + theme(
    #     figure_size=(8,5),
    # )
)
p

## All-Star

In [ ]:
df1 = pd.read_parquet("../data/misc/"+"NBA_All_Stars.parquet")
df1.columns = map(str.lower, df1.columns)

In [ ]:
df1.columns

In [ ]:
# award_list = ['All-NBA']
award_list = ['NBA All-Star']
df2 = df1[df1["description"].isin(award_list)]
cols = ['person_id' ,'season']
df2 = df2[cols]
df2 = df2.rename(columns={"person_id":"player_id"})
df2["season"] = df2["season"].str[:-3]
df2["season"] = df2["season"].astype(int) + 1
df2 = df2.sort_values(["player_id","season"])
pIDas = df2["player_id"].to_list()
df2["all_star"] = True

In [ ]:
dfa = []
for season in range(2010,2025):
    cols= ['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION','AGE', 'GP', ]
    df3 = pd.read_parquet(box_DIR + f"NBA_Box_P_Cum_Base_{season}.parquet",columns=cols)
    df3.columns = map(str.lower, df3.columns)
    df3["season"] = season + 1 
    df4 = pd.merge(df3,df2,on=["player_id","season"],how="left")
    dfa.append(df4)
df5 = pd.concat(dfa)
df5 = df5.sort_values(["player_id","season"]).reset_index(drop=True)

In [ ]:
df5["all_star"] = df5["all_star"].fillna(False)
df6 = df5[df5["player_id"].isin(pIDas)]
df6["count"]= np.where(df6["all_star"], 1, 0)
df6["wind"] = df6["count"].rolling(3).sum()
df6["star"] = df6["wind"] >1
df6 = df6.dropna().reset_index(drop=True)

In [ ]:
df7 = df6.query("wind > 0").reset_index(drop=True)
df7g = df7.groupby(["player_id","player_name"])
keys = list(df7g.groups)

In [ ]:
dfb = []
for key in keys:    
    df8 = df7g.get_group(key)    
    df8["team_id_n"] = df8["team_id"].shift(-1)
    df8 = df8.dropna()
    df8["team_id_n"] = df8["team_id_n"].astype(int)
    df8["team_change"] = df8["team_id_n"] != df8["team_id"]
    df8 = df8[df8["team_change"] & df8["star"]]
    dfb.append(df8)
df9 = pd.concat(dfb)
df9 = df9.reset_index(drop=True)
df9["T1"] = df9["team_id"].map(teams_dict)
df9["T2"] = df9["team_id_n"].map(teams_dict)
df9 = df9.drop(columns=["team_abbreviation","team_id","team_id_n","team_change","all_star","count","player_id"])

In [ ]:
# Since 2016
df9 = df9.query("season > 2015").reset_index(drop=True)
plist = ["DeMarcus Cousins","Isaiah Thomas","Victor Oladipo","Ben Simmons"]
slist = [2019,2018,2020,2021]
for p,s in zip(plist,slist):
    try:
        df9 = df9.drop(index=df9.loc[(df9["player_name"] == p) & (df9["season"] == s)].index[0])
    except:
        continue
df10 = df9.reset_index(drop=True)

In [ ]:
df11 = df10.groupby("season")[["player_name","gp","age"]].agg({"player_name":"count","gp":"mean","age":"mean"})
df11 = df11.reset_index()
df11["gp"] = df11["gp"].round(1)
df11["age"] = df11["age"].round(1)
df11["player_name"] = df11["player_name"].astype(int)
df11.columns = ["Season","Players","Games Played","Age"]

In [ ]:
data = df11
p = (
    ggplot(data)
    + aes(x="Season",y="Players",group=0)
    # + geom_col(show_legend=False, position=position_stack(reverse=True),color="white")
    + geom_line(size=1.2)
    + geom_point(size=1.5)
    + theme_idv
    + labs(
        title = "Number of Star Players Changing Teams",
        subtitle = "Star: Player Making All-Star Team in 2 of previous 3 seasons",
        caption="bsky:@sradjoker.cc | x:@sradjoker | source: nba.com/stats",
    )
    # + theme(
    #     figure_size=(8,5),
    # )
)
p

In [ ]:
data = df11
p = (
    ggplot(data)
    + aes(x="Season",y="Age",group=0)
    # + geom_col(show_legend=False, position=position_stack(reverse=True),color="white")
    + geom_line(size=1.2)
    + geom_point(size=1.5)
    + theme_idv
    + labs(
        title = "Number of Star Players Changing Teams",
        subtitle = "Star: Player Making All-Star Team in 2 of previous 3 seasons",
        caption="bsky:@sradjoker.cc | x:@sradjoker | source: nba.com/stats",
    )
    # + theme(
    #     figure_size=(8,5),
    # )
)
p

In [ ]:
data = df11
p = (
    ggplot(data)
    + aes(x="Season",y="Games Played",group=0)
    # + geom_col(show_legend=False, position=position_stack(reverse=True),color="white")
    + geom_line(size=1.2)
    + geom_point(size=1.5)
    + theme_idv
    + labs(
        title = "Number of Star Players Changing Teams",
        subtitle = "Star: Player Making All-Star Team in 2 of previous 3 seasons",
        caption="bsky:@sradjoker.cc | x:@sradjoker | source: nba.com/stats",
    )
    # + theme(
    #     figure_size=(8,5),
    # )
)
p

## All-NBA vs All-Star

In [ ]:
df1 = pd.read_parquet("../data/misc/"+"NBA_All_Stars.parquet")
df1.columns = map(str.lower, df1.columns)

In [ ]:
award_list = ['All-NBA','NBA All-Star']
df2 = df1[df1["description"].isin(award_list)]
cols = ['person_id', 'first_name', 'last_name', 'team', 'description', 'season']
df2 = df2[cols]
df2["first_name"] = df2["first_name"] + " " + df2["last_name"]
df2 = df2.rename(columns={"first_name":"name"})
df2 = df2.drop(columns={"last_name"})
df2 = df2.rename(columns={"person_id":"player_id"})
df2["season"] = df2["season"].str[:-3]
df2["season"] = df2["season"].astype(int) + 1
df2 = df2.sort_values(["player_id","season"])
# pIDas = df2["player_id"].to_list()
# df2["all_star"] = True

In [ ]:
df2["all_star"] = np.where(df2["description"] == 'NBA All-Star', 1, 0)
df2["all_nba"] =  np.where(df2["description"] == 'All-NBA', 1, 0)
df3 = df2.groupby(["player_id","name","season"])[["all_star","all_nba"]].agg({"all_star":sum,"all_nba":sum})
df3 = df3.reset_index()
df4 = df3.groupby(["player_id","name"])[["all_star","all_nba"]].agg({"all_star":sum,"all_nba":sum})
df4 = df4.reset_index()
df4 = df4.sort_values("all_star",ascending=False).reset_index(drop=True)

In [ ]:
pIDl = data["player_id"].to_list()

In [ ]:
data = df4.head(20)
data["image"] = img_DIR_P + data["player_id"].astype(str) + ".png"
p = (
    ggplot(data)
    + aes(x="all_star",y="all_nba",image="image")
    + geom_image(size=0.2)
    + theme_idv
    + labs(
        title = "All-Star vs All-NBA",
        # subtitle = "Star: Player Making All-Star Team in 2 of previous 3 seasons",
        caption="bsky:@sradjoker.cc | x:@sradjoker | source: nba.com/stats",
    )
)
p